In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
raw_csv_data = np.loadtxt('Audiobooks-data.csv', delimiter=',')
unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:, -1]

# Balancing the Data

In [3]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []
for i in range(targets_all.shape[0]):
    if targets_all[i] ==0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

# Standarized Data

In [4]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

# Shaffle the data

In [5]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

# Split the data set into Train, Validation and Test

In [6]:
samples_count = shuffled_inputs.shape[0]
train_samples_count = int(0.8* samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]


test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

In [7]:
print(np.sum(train_targets), train_samples_count, np.sum(train_targets)/train_samples_count)
print()
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets)/validation_samples_count)
print()
print(np.sum(test_targets), test_samples_count, np.sum(test_targets)/test_samples_count)

1787.0 3579 0.4993014808605756

217.0 447 0.4854586129753915

233.0 448 0.5200892857142857


# Sve the three data in npz

In [8]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

# create the machine learning algorithm

## Import libraries

In [9]:
import numpy as np
import tensorflow as tf

# Data

In [10]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['inputs'].astype(np.int)

# Model

In [11]:
input_size = 10
output_size = 2
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50
    
# define how the model will look like
model = tf.keras.Sequential([
    
    # the first layer (the input layer)
    # each observation is 28x28x1 pixels, therefore it is a tensor of rank 3
    # since we don't know CNNs yet, we don't know how to feed such input into our net, so we must flatten the images
    # there is a convenient method 'Flatten' that simply takes our 28x28x1 tensor and orders it into a (None,) 
    # or (28x28x1,) = (784,) vector
    # this allows us to actually create a feed forward neural network
    
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
batch_size = 100
max_epochs = 100
model.fit(train_inputs,
         train_targets,
         batch_size=batch_size,
         epochs=max_epochs,
         validation_data=(validation_inputs, validation_targets),
         verbose=2)

Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 0s - loss: 0.6169 - accuracy: 0.6530 - val_loss: 0.5160 - val_accuracy: 0.7181
Epoch 2/100
3579/3579 - 0s - loss: 0.4653 - accuracy: 0.7692 - val_loss: 0.4395 - val_accuracy: 0.7763
Epoch 3/100
3579/3579 - 0s - loss: 0.4126 - accuracy: 0.7913 - val_loss: 0.4057 - val_accuracy: 0.7808
Epoch 4/100
3579/3579 - 0s - loss: 0.3880 - accuracy: 0.7944 - val_loss: 0.3919 - val_accuracy: 0.7942
Epoch 5/100
3579/3579 - 0s - loss: 0.3727 - accuracy: 0.8027 - val_loss: 0.3792 - val_accuracy: 0.7875
Epoch 6/100
3579/3579 - 0s - loss: 0.3627 - accuracy: 0.8106 - val_loss: 0.3794 - val_accuracy: 0.7897
Epoch 7/100
3579/3579 - 0s - loss: 0.3550 - accuracy: 0.8117 - val_loss: 0.3673 - val_accuracy: 0.7919
Epoch 8/100
3579/3579 - 0s - loss: 0.3479 - accuracy: 0.8122 - val_loss: 0.3651 - val_accuracy: 0.7964
Epoch 9/100
3579/3579 - 0s - loss: 0.3435 - accuracy: 0.8164 - val_loss: 0.3655 - val_accuracy: 0.8054
Epoch 10/100
3579/3579 - 0

Epoch 80/100
3579/3579 - 0s - loss: 0.3011 - accuracy: 0.8365 - val_loss: 0.3438 - val_accuracy: 0.8188
Epoch 81/100
3579/3579 - 0s - loss: 0.3021 - accuracy: 0.8393 - val_loss: 0.3498 - val_accuracy: 0.7919
Epoch 82/100
3579/3579 - 0s - loss: 0.3011 - accuracy: 0.8385 - val_loss: 0.3396 - val_accuracy: 0.8188
Epoch 83/100
3579/3579 - 0s - loss: 0.3010 - accuracy: 0.8391 - val_loss: 0.3389 - val_accuracy: 0.8233
Epoch 84/100
3579/3579 - 0s - loss: 0.3028 - accuracy: 0.8349 - val_loss: 0.3451 - val_accuracy: 0.7897
Epoch 85/100
3579/3579 - 0s - loss: 0.3024 - accuracy: 0.8365 - val_loss: 0.3494 - val_accuracy: 0.8098
Epoch 86/100
3579/3579 - 0s - loss: 0.3029 - accuracy: 0.8363 - val_loss: 0.3526 - val_accuracy: 0.8054
Epoch 87/100
3579/3579 - 0s - loss: 0.3068 - accuracy: 0.8363 - val_loss: 0.3462 - val_accuracy: 0.8009
Epoch 88/100
3579/3579 - 0s - loss: 0.3036 - accuracy: 0.8301 - val_loss: 0.3485 - val_accuracy: 0.7919
Epoch 89/100
3579/3579 - 0s - loss: 0.2963 - accuracy: 0.8421 - 